In [56]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import PIL
import torch

In [4]:
metadata = pd.read_csv('metadata.csv')

In [10]:
metadata.head(10)

,img_id,img_filename,y,split,place,place_filename
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,2,1,/o/ocean/00002178.jpg
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/l/lake/natural/00000065.jpg
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1,2,0,/b/bamboo_forest/00000131.jpg
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/o/ocean/00001268.jpg
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/o/ocean/00003147.jpg
5,6,001.Black_footed_Albatross/Black_Footed_Albatr...,1,2,1,/l/lake/natural/00002698.jpg
6,7,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/o/ocean/00003917.jpg
7,8,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1,1,/o/ocean/00001505.jpg
8,9,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/l/lake/natural/00000560.jpg
9,10,001.Black_footed_Albatross/Black_Footed_Albatr...,1,2,1,/o/ocean/00002073.jpg


In [39]:
len(metadata[metadata['split'] == 0])

4795

In [8]:
# split 0 is train, 1 is val, 2 is test
metadata['split'].value_counts()

2    5794
0    4795
1    1199
Name: split, dtype: int64

In [18]:
import shutil
import os

# copy files to train, val, test folders
for i in range(len(metadata)):
    if metadata['split'][i] == 2:
        #shutil.copy(metadata['image_id'][i] + '.jpg', 'train/' + metadata['image_id'][i] + '.jpg')
        shutil.copy(metadata['img_filename'][i], 'test/' + str(metadata['img_id'][i]) + '.jpg')
    

In [48]:
metadata.iloc[5]

img_id                                                            6
img_filename      001.Black_footed_Albatross/Black_Footed_Albatr...
y                                                                 1
split                                                             2
place                                                             1
place_filename                         /l/lake/natural/00002698.jpg
Name: 5, dtype: object

In [67]:
# create a custom training dataset class
class WaterbirdsDataset(Dataset):
    def __init__(self, csv_dir, root_dir = "./", mode="train", transform=None):
        metadata = pd.read_csv(csv_dir)
        if mode == "test":
            self.metadata = metadata[metadata['split'] == 2]
        elif mode == "val":
            self.metadata = metadata[metadata['split'] == 1]
        else:
            self.metadata = metadata[metadata['split'] == 0]
        
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.metadata.iloc[idx, 1])
        image = PIL.Image.open(img_name)
        label = self.metadata.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        image = transforms.ToTensor()(image)
        image = transforms.CenterCrop(224)(image)
        return image, label

In [70]:
dset= WaterbirdsDataset(csv_dir = 'metadata.csv', mode = 'val')
dset.__len__()
dataloader = DataLoader(dset, batch_size=32, shuffle=True)
it = iter(dataloader)
images, labels = next(it)

torch.Size([32, 3, 224, 224])